In [ ]:
import sys
import urllib.request
import numpy as np
sys.path.insert(0, '/Users/joaquimteixeira/GSR_22_Tansey_Banerjee/bayestme-main/src')
from bayestme import data
from sklearn.model_selection import train_test_split
from sklearn import decomposition
import pandas as pd

In [76]:
import os 
fls=os.listdir('/Users/joaquimteixeira/GSR_22_Tansey_Banerjee/teixeira_code/flipped_cohort_data/')
fout=[]
for k in fls:
    fout.append (k[:2])
fout

['A1', 'F1', 'G1', 'D1', 'H1', 'E1', 'B1', 'C1']

In [77]:
p=0
for f in fls:
    print (f)
    stdata = data.SpatialExpressionDataset.read_count_mat('/Users/joaquimteixeira/GSR_22_Tansey_Banerjee/teixeira_code/flipped_cohort_data/'+f)
    from bayestme import gene_filtering

    stddev_filtered = gene_filtering.select_top_genes_by_standard_deviation(
        stdata, n_gene=1000)

    spot_threshold_filtered = gene_filtering.filter_genes_by_spot_threshold(
        stddev_filtered, spot_threshold=0.95)

    stdata_filtered = gene_filtering.filter_ribosome_genes(spot_threshold_filtered)
    import os
    from bayestme import phenotype_selection
    from pathlib import Path

    Path("./cohort_k_fold_results").mkdir(exist_ok=True)


    parameter_generator = phenotype_selection.get_phenotype_selection_parameters_for_folds(
        stdata=stdata_filtered,
        n_fold=5,
        n_splits=15,
        lams=[10, 100, 1000,10000,100000,1000000],
        n_components_min=2,
        n_components_max=10
    )


    for job_index, (lam, n_components_for_job, mask, fold_number) in enumerate(parameter_generator):
        result: data.PhenotypeSelectionResult = w.run_phenotype_selection_single_job(
                lam=lam,
                n_components=n_components_for_job,
                mask=mask,
                fold_number=fold_number,
                stdata=stdata,
                n_samples=100,
                n_burn=2000,
                n_thin=5,
                max_ncell=100,
                n_gene=1000,
                background_noise=False,
                lda_initialization=False
        )
        result.save(os.path.join('./cohort_k_fold_results', 'fold_{}.h5ad'.format(job_index)))
    from bayestme import cv_likelihoods
    from IPython.display import Image


    likelihoods, fold_nums, lam_vals, k_vals = cv_likelihoods.load_likelihoods('./cohort_k_fold_results')

    best_n_components = cv_likelihoods.get_max_likelihood_n_components(likelihoods, k_vals)
    print (best_n_components)

    best_lambda = cv_likelihoods.get_best_lambda_value(
        likelihoods,
        best_n_components,
        lam_vals,
        k_vals)

    cv_likelihoods.plot_cv_running('./cohort_k_fold_results', out_path='.', output_format='png')
    from bayestme import deconvolution

    deconvolution_result = deconvolution.deconvolve(
            reads=stdata_filtered.reads,
            edges=stdata_filtered.edges,
            n_gene=931,
            n_components=best_n_components,
            lam2=best_lambda,
            n_samples=1000,
            n_burnin=2000,
            n_thin=5,
            bkg=False,
            lda=False)
    deconvolution_result.save('/Users/joaquimteixeira/GSR_22_Tansey_Banerjee/teixeira_code/deconvolution_results/'+fout[p]+'.h5ad')
    p=p+1


A1.tsv


/Users/joaquimteixeira/GSR_22_Tansey_Banerjee/bayestme-main/src/bayestme/data.py:58: FutureWarning: X.dtype being converted to np.float32 from int64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  adata = anndata.AnnData(counts, obsm={SPATIAL_ATTR: coordinates})


KeyboardInterrupt: 